In [1]:
from __future__ import division, print_function
import sys
import os
import glob
import re
from pathlib import Path
from io import BytesIO
import base64
import requests
import base64


# Import fast.ai Library
import torch
from fastai2.basics import *
from fastai2.callback.all import *
from fastai2.vision.all import *
from fastai2.vision.core import *
from fastai2.vision.widgets import *

# Flask utils
from flask import Flask, redirect, url_for, render_template, request
#from PIL import Image as pilImage

In [2]:

# Define a flask app
app = Flask(__name__)

NAME_OF_FILE = 'export.pkl' # Name of your exported file
PATH_TO_MODELS_DIR = Path() # by default just use /models in root dir
classes = ['Negative','Tumor']

In [3]:

# Include functions from model class
def get_x(r): 
    pathstr = str(path)
    idstr = str(r['id'])
    imgpathstr = pathstr + '/train/' + idstr + '.tif'
    return imgpathstr
def get_y(r): return r['label']

def setup_model_pth(path_to_pth_file, learner_name_to_load, classes):
    learn = load_learner(path_to_pth_file/'export.pkl')
    return learn

In [4]:
learn = setup_model_pth(PATH_TO_MODELS_DIR, NAME_OF_FILE, classes)

In [43]:
def change_pred_totext(preds):
    if (preds==1):
        predvalue = 'Positive'
    else:
        predvalue = 'Negative'
    return predvalue

def encode(img):
    img = Image.open(BytesIO(img))
    img = img.resize((500,500))
    buff = BytesIO()
    img.save(buff, format='jpeg')
    return base64.b64encode(buff.getvalue()).decode("utf-8")

# def pred_probs():
#     preds = sorted(zip(learn.dls.vocab, map(str, formatted_outputs)), key=lambda p: p[1], reverse=True
#     return preds    
	
def model_predict(img):
    #img = Image.open(BytesIO(img))
    pred_class,pred_idx,outputs = learn.predict(img)
    formatted_outputs = ["{:.1f}%".format(value) for value in [x * 100 for x in torch.nn.functional.softmax(outputs, dim=0)]]
    pred_probs = sorted(
            zip(learn.dls.vocab, map(str, formatted_outputs)),
            key=lambda p: p[1],
            reverse=True
        )
#     predvocab = learn.dls.vocab
#     preds = sorted(zip(predvocab, map(str, formatted_outputs)))
#     pred_probs = preds
    #img_data = encode(img)
    pred_class = change_pred_totext(pred_class)
    
    #Different return statement for debug of function
    result = {"class":pred_class, "probs":pred_probs}
    return result
    
    #img_data = encode(img)
    #result = {"class":pred_class, "probs":pred_probs, "image":img_data}
    #    return render_template('result.html', result=result)
    
  

In [44]:
path = Path()
#imgsrc = '/workspace/histopathologic-cancer-detection/train/6961bdcc16f6c1d7db88fc6a7823178288c2a29e.tif'
imgsrc = '/workspace/skincancer_fastaiflask/data/ham10000_images/ISIC_0024306.jpg'
#img = PILImage.create(img)

In [45]:
preds = model_predict(imgsrc)
print(preds)

{'class': 'Negative', 'probs': [(0, '73.0%'), (1, '27.0%')]}


In [ ]:
   

@app.route('/', methods=['GET', "POST"])
def index():
    # Main page
    return render_template('index.html')


@app.route('/upload', methods=["POST", "GET"])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        img = request.files['file'].read()
        if img != None:
        # Make prediction
            preds = model_predict(img)
            return preds
    return 'OK'
	
@app.route("/classify-url", methods=["POST", "GET"])
def classify_url():
    if request.method == 'POST':
        url = request.form["url"]
        if url != None:
            response = requests.get(url)
            preds = model_predict(response.content)
            return preds
    return 'OK'
    

In [ ]:
if __name__ == '__main__':
    port = os.environ.get('PORT', 8008)

    if "prepare" not in sys.argv:
        app.run(debug=True, host='0.0.0.0', port=port)


In [ ]:
%tb